# Módulo 3: Tratamento e Carga dos Dados no Banco de Dados (ETL)

## 1. O Ponto de Partida: Um Dataset Completo, Mas "Plano"

Nos módulos anteriores, nossos robôs de web scraping trabalharam para criar um arquivo CSV completo, contendo uma vasta quantidade de informações sobre anúncios do Airbnb. Já temos os dados de busca (Módulo 2) e os detalhes de cada imóvel (Módulo 1).

### O Desafio
Apesar de completo, nosso dataset está em um formato "plano" (uma única grande tabela). Para análises eficientes e para a criação de aplicações robustas, este formato não é ideal. Ele contém muita informação repetida (como o nome da cidade em todas as linhas) e não está otimizado para consultas complexas.

### A Solução: Normalização e Carga no Banco de Dados
Este notebook executa o processo de **ETL (Extract, Transform, Load)**:
1.  **Extract (Extrair):** Carregamos os dados brutos do nosso arquivo CSV consolidado.
2.  **Transform (Transformar):** Limpamos e transformamos os dados. O passo mais importante aqui é a **normalização**, onde quebramos a grande tabela em várias tabelas menores e relacionadas (`cidade`, `bairro`, `imovel`, etc.), cada uma com uma responsabilidade única. Isso elimina a redundância e cria um modelo de dados relacional e eficiente.
3.  **Load (Carregar):** Inserimos cada uma dessas novas tabelas tratadas em nosso banco de dados PostgreSQL, que servirá como nossa fonte de verdade centralizada e otimizada para futuras consultas.

## 2. Preparando o Ambiente: Ferramentas de Dados e Conexão

Para esta etapa, nossa "caixa de ferramentas" é mais focada. Precisamos do **Pandas** para a manipulação dos dados e das nossas funções customizadas do módulo `banco_de_dados`, que cuidam de toda a complexidade de se conectar ao banco e inserir os dados de forma eficiente.

In [ ]:
# Importa a biblioteca pandas, essencial para manipulação de dados em formato de tabela (DataFrame).
import pandas as pd

# Importa as funções personalizadas que criamos para interagir com o banco de dados.
# insere_dados_no_banco: envia um DataFrame para uma tabela no banco de dados.
# retorna_tabela: busca todos os dados de uma tabela do banco e os retorna como um DataFrame.
from funcoes.banco_de_dados import insere_dados_no_banco, retorna_tabela

## 3. Extract: Carregando o Dataset Bruto

A primeira etapa do nosso ETL é a **extração**. Vamos carregar o arquivo CSV, que é o resultado de todo o trabalho dos nossos scrapers, para a memória em um DataFrame do Pandas. A partir daqui, todas as transformações serão feitas neste DataFrame.

In [ ]:
# Lê o arquivo CSV contendo os dados brutos e completos e o carrega em um DataFrame chamado 'dados_brutos'.
dados_brutos = pd.read_csv('dados/todos_dados/dados_completos_rio_de_janeiro.csv')

# Exibe as primeiras linhas do DataFrame para uma verificação inicial dos dados.
dados_brutos.head()

## 4. Transform: Limpeza e Normalização dos Dados

Esta é a etapa mais crítica do processo. Aqui, vamos limpar e transformar nosso grande e único DataFrame em várias tabelas menores e organizadas, prontas para serem inseridas no banco de dados relacional. Vamos criar uma tabela para cada entidade principal: `cidade`, `bairro`, `imovel`, `avaliacao`, `agendamento` e `anuncio`.

### 4.1. Engenharia de Localização
Primeiro, vamos tratar a coluna `Localização`. Ela contém o bairro e a cidade juntos. Vamos separá-los em colunas distintas para podermos criar nossas tabelas `cidade` e `bairro`. Também adicionaremos a coluna `estado`.

In [ ]:
# Utiliza o acessador '.str.split()' para dividir a coluna 'Localização' em duas novas colunas ('bairro' e 'cidade').
# O delimitador usado para a separação é ', ' (vírgula seguida de espaço).
# 'expand=True' garante que o resultado seja expandido em novas colunas no DataFrame.
dados_brutos[['bairro', 'cidade']] = dados_brutos['Localização'].str.split(', ', expand=True)

# Aplica a função '.str.strip()' para remover quaisquer espaços em branco no início ou no fim das novas colunas.
# Isso garante a consistência dos dados (ex: " Copacabana" se torna "Copacabana").
dados_brutos['bairro'] = dados_brutos['bairro'].str.strip()
dados_brutos['cidade'] = dados_brutos['cidade'].str.strip()

# Cria uma nova coluna chamada 'estado' e atribui o valor fixo 'RJ' para todas as linhas.
# Isso é feito pois todos os dados coletados neste projeto são do Rio de Janeiro.
dados_brutos['estado'] = 'RJ'

### 4.2. Preparando e Carregando a Tabela `cidade`

A tabela `cidade` é a mais simples. Ela conterá apenas uma lista de cidades únicas e seus respectivos estados. Isso evita que a gente repita "Rio de Janeiro" e "RJ" milhões de vezes no nosso banco de dados.

O processo é:
1.  Selecionar as colunas `cidade` e `estado`.
2.  Remover as duplicatas, deixando apenas uma linha por cidade.
3.  Renomear a coluna `cidade` para `nome`, para bater com o nosso modelo do banco.
4.  Inserir no banco de dados.
5.  Recuperar os dados do banco (com os IDs gerados) para usar nas próximas etapas.

In [ ]:
# --- 1. Preparação do DataFrame 'cidades' ---
# Seleciona apenas as colunas 'cidade' e 'estado' do nosso DataFrame principal.
cidades = dados_brutos[['cidade', 'estado']]
# Remove todas as linhas duplicadas, resultando em uma lista de cidades únicas.
cidades = cidades.drop_duplicates()
# Renomeia a coluna 'cidade' para 'nome' para corresponder ao esquema da tabela no banco de dados. 'inplace=True' aplica a mudança diretamente.
cidades.rename({'cidade':'nome'}, axis=1, inplace=True)

# --- 2. Carga no Banco de Dados ---
# Chama nossa função personalizada para inserir o DataFrame 'cidades' na tabela 'cidade' do banco.
insere_dados_no_banco(cidades, 'cidade')

# --- 3. Verificação e Recuperação ---
# Chama a função para buscar os dados recém-inseridos na tabela 'cidade'.
# Agora, o DataFrame 'cidades' terá a coluna 'id' gerada automaticamente pelo banco, que usaremos como chave estrangeira.
cidades = retorna_tabela('cidade')
# Exibe o DataFrame recuperado para confirmar que a inserção funcionou.
cidades

### 4.3. Preparando e Carregando a Tabela `bairro`

Agora, faremos o mesmo para os bairros. A diferença crucial aqui é que precisamos relacionar cada bairro à sua cidade correspondente. Para isso, usaremos o `id` da cidade que acabamos de inserir no banco.

In [ ]:
# --- 1. Preparação do DataFrame 'bairros' ---
# Seleciona as colunas 'bairro' e 'cidade' e remove as combinações duplicadas.
bairros = dados_brutos[['bairro', 'cidade']].drop_duplicates()
# Renomeia a coluna 'bairro' para 'nome'.
bairros.rename({'bairro':'nome'}, axis=1, inplace=True)

# --- 2. Adicionando a Chave Estrangeira (cidade_id) ---
# Usa a função 'pd.merge' para juntar o DataFrame de bairros com o de cidades (que já tem o 'id').
# A junção é feita onde a coluna 'cidade' do df 'bairros' é igual à coluna 'nome' do df 'cidades'.
bairrost = pd.merge(bairros, cidades, left_on='cidade', right_on='nome')
# Renomeia as colunas resultantes para o padrão do banco ('id' vira 'cidade_id').
bairrost.rename({'nome_x':'nome', 'id':'cidade_id'}, axis=1, inplace=True)
# Seleciona apenas as colunas finais que serão inseridas na tabela 'bairro'.
bairrost = bairrost[['nome', 'cidade_id']]

# --- 3. Carga no Banco de Dados ---
insere_dados_no_banco(bairrost, 'bairro')

# --- 4. Verificação e Recuperação ---
# Busca a tabela 'bairro' do banco para obter os IDs de cada bairro.
bairros = retorna_tabela('bairro')
# Exibe o resultado.
bairros

### 4.4. Preparando e Carregando a Tabela `imovel`

A tabela `imovel` conterá as características estáticas de cada propriedade única (tipo, quartos, camas, etc.). O processo envolve:
1.  Selecionar as colunas relevantes.
2.  Remover duplicatas baseadas no `ID Imóvel` para ter uma linha por imóvel.
3.  Juntar (merge) com a tabela de bairros que acabamos de criar para obter a `bairro_id`.
4.  Limpar os dados de quartos/camas/banheiros para conter apenas números.
5.  Carregar no banco.

In [ ]:
# --- 1. Preparação do DataFrame 'imovel' ---
# Seleciona as colunas com características fixas do imóvel.
imovel = dados_brutos[['ID Imóvel', 'Tipo de Acomodação', 'Quartos', 'Camas', 'Banheiros', 'bairro']]
# Remove duplicatas baseadas na coluna 'ID Imóvel' para garantir que cada imóvel apareça apenas uma vez.
imovel = imovel.drop_duplicates('ID Imóvel')
# Renomeia as colunas para o padrão do banco.
imovel.rename({'ID Imóvel':'id_imovel', 'Tipo de Acomodação':'tipo_acomodacao'}, axis=1, inplace=True)

# --- 2. Adicionando a Chave Estrangeira (bairro_id) ---
# Junta o DataFrame 'imovel' com o DataFrame 'bairros' (já com IDs do banco) para obter a 'bairro_id' de cada imóvel.
imovelt = pd.merge(imovel, bairros, left_on='bairro', right_on='nome')
# Renomeia as colunas para o padrão do banco.
imovelt.rename({'id':'bairro_id','Quartos':'quartos' ,'Camas':'camas', 'Banheiros':'banheiros'}, axis=1, inplace=True)
# Remove colunas que foram usadas apenas para a junção e não são necessárias na tabela final.
imovelt = imovelt.drop(columns=['nome','bairro'])
# Adiciona a 'cidade_id' diretamente, pois sabemos que todos os bairros pertencem à mesma cidade neste caso.
imovelt['cidade_id'] = cidades['id'].iloc[0]


# --- 3. Limpeza Final dos Dados ---
# Itera sobre as colunas 'quartos', 'camas' e 'banheiros'.
for col in ['quartos', 'camas', 'banheiros']:
  # Usa regex para extrair apenas o primeiro conjunto de dígitos de cada string (ex: '2 camas' vira '2').
  imovelt[col] = imovelt[col].str.extract('(\\d+)')

# --- 4. Carga no Banco de Dados ---
insere_dados_no_banco(imovelt, 'imovel')

# --- 5. Verificação e Recuperação ---
# Busca a tabela 'imovel' do banco para usá-la nas próximas junções.
imovel = retorna_tabela('imovel')
imovel

### 4.5. Preparando e Carregando a Tabela `avaliacao`

Esta tabela isola as informações de avaliação, que podem variar para um mesmo imóvel.

1.  Selecionar as colunas de avaliação.
2.  Juntar com a tabela `imovel` para obter a `imovel_id` (chave estrangeira).
3.  Limpar e converter os dados de `nota` e `qtd_avaliacoes` para os tipos numéricos corretos.
4.  Carregar no banco.

In [ ]:
# --- 1. Preparação do DataFrame 'avaliacao' ---
# Seleciona as colunas relevantes e remove duplicatas.
avaliacao = dados_brutos[['Quantidade de Avaliações', 'Avaliação', 'ID Imóvel']].drop_duplicates()
# Renomeia as colunas para o padrão do banco.
avaliacao.rename({'Quantidade de Avaliações':'qtd_avaliacoes', 'Avaliação':'nota', 'ID Imóvel': 'imovel_id'}, axis=1, inplace=True)

# --- 2. Adicionando a Chave Estrangeira (imovel_id) ---
# Junta o DataFrame 'avaliacao' com 'imovel' (do banco) para obter o ID numérico correto para cada imóvel.
avaliacaot = pd.merge(avaliacao, imovel[['id', 'id_imovel']], left_on='imovel_id', right_on='id_imovel')
# Remove colunas de ID redundantes.
avaliacaot = avaliacaot.drop(columns=['imovel_id', 'id_imovel'])
# Renomeia a coluna 'id' (vinda da tabela imovel) para 'imovel_id', que é a chave estrangeira correta.
avaliacaot.rename({'id':'imovel_id'}, axis=1, inplace=True)

# --- 3. Limpeza Final dos Dados ---
# Preenche valores nulos (NaN) na quantidade de avaliações com 0.
avaliacaot['qtd_avaliacoes'] = avaliacaot['qtd_avaliacoes'].fillna(0)
# Converte a coluna para o tipo inteiro.
avaliacaot['qtd_avaliacoes'] = avaliacaot['qtd_avaliacoes'].astype(int)
# Na coluna 'nota', substitui a vírgula por ponto para permitir a conversão para número.
avaliacaot['nota'] = avaliacaot['nota'].str.replace(',','.')
# Substitui o texto 'Novo' (para anúncios sem avaliação) por 0.
avaliacaot['nota'] = avaliacaot['nota'].replace('Novo',0)
# Preenche quaisquer outros valores nulos com 0.
avaliacaot['nota'] = avaliacaot['nota'].fillna(0)
# Converte a coluna para o tipo float (número com casas decimais).
avaliacaot['nota'] = avaliacaot['nota'].astype(float)

# --- 4. Carga no Banco de Dados ---
insere_dados_no_banco(avaliacaot, 'avaliacao')

### 4.6. Preparando e Carregando a Tabela `agendamento`

A tabela `agendamento` guarda as informações que mudam a cada busca: datas, preços para aquele período e número de hóspedes. É uma das tabelas mais dinâmicas do nosso modelo.

In [ ]:
# --- 1. Preparação do DataFrame 'agendamento' ---
# Seleciona as colunas relevantes para o agendamento.
agendamento = dados_brutos[['Número de Hóspedes', 'Data de Check-in', 'Data de Check-out', 'ID Imóvel', 'Preço total', 'Link']]
# Remove duplicatas para ter apenas uma combinação única de imóvel-datas-preço.
agendamento = agendamento.drop_duplicates()

# --- 2. Limpeza e Transformação dos Dados ---
# Converte as colunas de data (que estão como texto) para o tipo datetime do pandas. 'dayfirst=True' informa que o dia vem antes do mês (formato DD/MM/AAAA).
agendamento['data_checkin'] = pd.to_datetime(agendamento['Data de Check-in'], dayfirst=True)
agendamento['data_checkout'] = pd.to_datetime(agendamento['Data de Check-out'], dayfirst=True)
# Calcula a duração da estadia subtraindo as datas e extraindo o número de dias.
agendamento['total_noites'] = (agendamento['data_checkout'] - agendamento['data_checkin']).dt.days.astype(int)
# Limpa a coluna de preço, removendo 'R$', espaços, e convertendo para número inteiro.
agendamento['preco_total'] = agendamento['Preço total'].str.replace('R$', '', regex=False).str.strip().astype(float).astype(int)
# Calcula o preço médio por dia.
agendamento['preco_por_dia'] = (agendamento['preco_total'] / agendamento['total_noites']).round(2)

# Renomeia as colunas para o padrão do banco.
agendamento.rename({'Número de Hóspedes':'hospedes', 'Link':'link'}, axis=1, inplace=True)

# --- 3. Adicionando a Chave Estrangeira (imovel_id) ---
# Junta com a tabela 'imovel' para obter o ID correto de cada imóvel.
agendamentot = pd.merge(agendamento, imovel[['id_imovel', 'id']], left_on='ID Imóvel', right_on='id_imovel')
# Renomeia a coluna 'id' para 'imovel_id'.
agendamentot.rename({'id':'imovel_id'}, axis=1, inplace=True)
# Seleciona e reordena as colunas finais para inserção, removendo as colunas de junção.
agendamentot = agendamentot[['hospedes', 'data_checkin', 'data_checkout', 'preco_por_dia','preco_total', 'link', 'imovel_id']]

# --- 4. Carga no Banco de Dados ---
insere_dados_no_banco(agendamentot,'agendamento')

### 4.7. Preparando e Carregando a Tabela `anuncio`

Finalmente, a tabela `anuncio` é a nossa "tabela fato". Ela conecta todas as outras, ligando um `imovel` a um `agendamento` específico através de suas chaves estrangeiras. Cada linha aqui representa uma oferta única que foi encontrada em nossas buscas.

In [ ]:
# --- 1. Recuperação e Preparação dos Dados ---
# Primeiro, recuperamos a tabela 'agendamento' que acabamos de inserir, pois ela agora contém o 'id' único para cada agendamento.
agendamento_db = retorna_tabela('agendamento')
# Converte as colunas de data para o tipo datetime para garantir a correspondência na junção.
agendamento_db['data_checkin'] = pd.to_datetime(agendamento_db['data_checkin'])
agendamento_db['data_checkout'] = pd.to_datetime(agendamento_db['data_checkout'])

# Prepara o DataFrame 'anuncio' a partir dos dados brutos.
anuncio = dados_brutos[['Tipo de Acomodação', 'Link', 'ID Imóvel', 'Data de Check-in', 'Data de Check-out', 'Número de Hóspedes']]
anuncio = anuncio.drop_duplicates()
anuncio.rename({'Tipo de Acomodação':'titulo', 'Link':'link', 'Data de Check-in':'data_checkin', 'Data de Check-out':'data_checkout', 'Número de Hóspedes':'hospedes'}, axis=1, inplace=True)
anuncio['data_checkin'] = pd.to_datetime(anuncio['data_checkin'], dayfirst=True)
anuncio['data_checkout'] = pd.to_datetime(anuncio['data_checkout'], dayfirst=True)

# --- 2. Adicionando as Chaves Estrangeiras ---
# Junta 'anuncio' com 'imovel' para obter a 'imovel_id' do banco.
anuncio_temp = pd.merge(anuncio, imovel[['id_imovel', 'id']], left_on='ID Imóvel', right_on='id_imovel')
anuncio_temp.rename({'id':'imovel_id'}, axis=1, inplace=True)

# Garante que os tipos das colunas de junção sejam os mesmos em ambos os DataFrames.
anuncio_temp['imovel_id'] = anuncio_temp['imovel_id'].astype(int)
agendamento_db['imovel_id'] = agendamento_db['imovel_id'].astype(int)

# Junta o resultado anterior com 'agendamento_db' para obter a 'agendamento_id'.
# A junção é complexa, usando múltiplas colunas para garantir que estamos ligando o anúncio à oferta exata.
anunciot = pd.merge(anuncio_temp, agendamento_db[['id', 'imovel_id', 'data_checkin', 'data_checkout', 'hospedes', 'link']], on=['imovel_id', 'data_checkin', 'data_checkout', 'hospedes', 'link'])
anunciot.rename({'id':'agendamento_id'}, axis=1, inplace=True)

# Seleciona as colunas finais para a tabela 'anuncio'.
anunciot = anunciot[['titulo', 'link', 'agendamento_id']]

# --- 3. Carga no Banco de Dados ---
insere_dados_no_banco(anunciot,'anuncio')

## 5. Conclusão do Processo de ETL

Com a execução da última célula, concluímos nosso pipeline de ETL. O que antes era um único e massivo arquivo CSV, agora é um conjunto de tabelas limpas, organizadas e inter-relacionadas dentro de um banco de dados PostgreSQL.

Os dados estão agora em um formato ideal para:
* **Consultas SQL complexas** para responder perguntas de negócio.
* **Conexão com ferramentas de Business Intelligence** (como Power BI, Looker, etc.) para criar dashboards.
* **Servir como fonte de dados para modelos de Machine Learning** e outras aplicações.

O projeto de dados pode agora avançar para a fase de análise e geração de valor.